In [1]:
# https://developer.spotify.com/console/get-audio-features-several-tracks/

In [2]:
import pandas as pd
import datetime as dt
from sqlalchemy import create_engine
import requests
import pymongo
import json

# pd.set_option('display.max_rows', None)
#pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
path = "../Resources/Charts CSV/regional-global-daily-latest.csv"
spotify_data_global = pd.read_csv(path)
spotify_data_global.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Note that these figures are generated using a formula that protects against any artificial inflation of chart positions.,Unnamed: 4
0,Position,Track Name,Artist,Streams,URL
1,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e
2,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
3,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
4,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE


In [4]:
spotify_data_global.to_csv("clean_global.csv", header=False, index=False)


In [5]:
path_clean = "clean_global.csv"
spotify_data_global_clean = pd.read_csv(path_clean)
spotify_data_global_clean.head()

,Position,Track Name,Artist,Streams,URL
0,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e
1,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20
2,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG
3,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE
4,5,Kiss Me More (feat. SZA),Doja Cat,4217120,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz


In [6]:
spotify_data_global_clean["ID"] = spotify_data_global_clean["URL"].str[31:]
spotify_data_global_clean.head()

,Position,Track Name,Artist,Streams,URL,ID
0,1,Beggin',Måneskin,7178321,https://open.spotify.com/track/3Wrjm47oTz2sjIgck11l5e,3Wrjm47oTz2sjIgck11l5e
1,2,Stay (with Justin Bieber),The Kid LAROI,6355637,https://open.spotify.com/track/5HCyWlXZPP0y6Gqq8TgA20,5HCyWlXZPP0y6Gqq8TgA20
2,3,good 4 u,Olivia Rodrigo,6177736,https://open.spotify.com/track/4ZtFanR9U6ndgddUvNcjcG,4ZtFanR9U6ndgddUvNcjcG
3,4,Bad Habits,Ed Sheeran,5475630,https://open.spotify.com/track/6PQ88X9TkUIAUIZJHW2upE,6PQ88X9TkUIAUIZJHW2upE
4,5,Kiss Me More (feat. SZA),Doja Cat,4217120,https://open.spotify.com/track/3DarAbFujv6eYNliUTyqtz,3DarAbFujv6eYNliUTyqtz


In [7]:
len("https://open.spotify.com/track/")

31

In [8]:
test = spotify_data_global_clean["URL"].str
print(test[31:])

0      3Wrjm47oTz2sjIgck11l5e
1      5HCyWlXZPP0y6Gqq8TgA20
2      4ZtFanR9U6ndgddUvNcjcG
3      6PQ88X9TkUIAUIZJHW2upE
4      3DarAbFujv6eYNliUTyqtz
                ...          
195    5EVzgDPaM0LCM3M5exl3z8
196    7fBv7CLKzipRk6EC6TWHOB
197    5zqObw7wjBgL9TDiAymxPn
198    3rq5w4bQGigXOfdN30ATJt
199    4JeOesjcWWlprTAvucDSJX
Name: URL, Length: 200, dtype: object


In [12]:
token = "BQDkUVHSHlnDbARSqmu8syVDQGhqAeZee8NYPICTtocLBkqD3N-h6ziZHUQVFVyi3N6wtI1lKgmlkwKB2nqq0GTkkDTxNkg1GdyrrUkCbe1g9x4UyaCT2vILcmGU5VzIS9QFcMHqFPJ0aqAyNKb1zcRFwwYgzcoUoLdvIdF1xLjQUYcub8Ey7UrahpmoWr5C7zhnkqV0m6Z4T7dmFoIARROaB8AzsxJD2tJHVef24fMmmice1OxfzHedCoi_KaTkTlT9-E5VZHfOKwGuR76T"
headers_dict = {"Authorization": f'Bearer {token}'}

# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.SpotifyTopChartsdb
collection = db.TopGlobal
collection.drop()

rows=len(spotify_data_global_clean)
global_dict={}
# spotify_artist_dict=[]

for row in range(0,rows):
    # Hacer el llamado a la API de Spotify usando spotify_data_wNA.iloc[row,4]
    artist_id = spotify_data_global_clean.iloc[row,5]
    artist_name = spotify_data_global_clean.iloc[row,2]
    track_name = spotify_data_global_clean.iloc[row,1]

    
    response = requests.get(f'https://api.spotify.com/v1/audio-features?ids={artist_id}', headers=headers_dict).json()
    # print(response)
    # artist_id = response["audio_features"]
    # print(response["audio_features"])

    danceability = response["audio_features"][0]["danceability"]
    energy = response["audio_features"][0]["energy"]
    key = response["audio_features"][0]["liveness"]
    loudness = response["audio_features"][0]["key"]
    acousticness = response["audio_features"][0]["acousticness"]
    instrumentalness = response["audio_features"][0]["instrumentalness"]
    liveness = response["audio_features"][0]["liveness"]
    valence = response["audio_features"][0]["valence"]
    tempo = response["audio_features"][0]["tempo"]
    track_href = response["audio_features"][0]["track_href"]
        
        
    global_dict={'Artist Name':artist_name,
                 'Track Name':track_name,
                 'Danceability': danceability,
                 'Energy': energy,
                 'Key': key,
                 'Loudness': loudness,
                 'Acousticness': acousticness,
                 'Instrumentalness': instrumentalness,
                 'Liveness': liveness,
                 'Valence': valence,
                 'Tempo': tempo,
                 'Track Reference': track_href
                 }
    collection.insert_one(global_dict)
#     spotify_artist_dict.append(artist_dict)
   
    print(global_dict)

{'Artist Name': 'Måneskin', 'Track Name': "Beggin'", 'Danceability': 0.714, 'Energy': 0.8, 'Key': 0.359, 'Loudness': 11, 'Acousticness': 0.127, 'Instrumentalness': 0, 'Liveness': 0.359, 'Valence': 0.589, 'Tempo': 134.002, 'Track Reference': 'https://api.spotify.com/v1/tracks/3Wrjm47oTz2sjIgck11l5e', '_id': ObjectId('60f789a3e867b24c8ca86ae7')}
{'Artist Name': 'The Kid LAROI', 'Track Name': 'Stay (with Justin Bieber)', 'Danceability': 0.591, 'Energy': 0.764, 'Key': 0.103, 'Loudness': 1, 'Acousticness': 0.0383, 'Instrumentalness': 0, 'Liveness': 0.103, 'Valence': 0.478, 'Tempo': 169.928, 'Track Reference': 'https://api.spotify.com/v1/tracks/5HCyWlXZPP0y6Gqq8TgA20', '_id': ObjectId('60f789a4e867b24c8ca86ae8')}
{'Artist Name': 'Olivia Rodrigo', 'Track Name': 'good 4 u', 'Danceability': 0.563, 'Energy': 0.664, 'Key': 0.0849, 'Loudness': 9, 'Acousticness': 0.335, 'Instrumentalness': 0, 'Liveness': 0.0849, 'Valence': 0.688, 'Tempo': 166.928, 'Track Reference': 'https://api.spotify.com/v1/trac

{'Artist Name': 'Masked Wolf', 'Track Name': 'Astronaut In The Ocean', 'Danceability': 0.778, 'Energy': 0.695, 'Key': 0.15, 'Loudness': 4, 'Acousticness': 0.175, 'Instrumentalness': 0, 'Liveness': 0.15, 'Valence': 0.472, 'Tempo': 149.996, 'Track Reference': 'https://api.spotify.com/v1/tracks/0BGwAKW4u8kWOhWFflZxfl', '_id': ObjectId('60f789a8e867b24c8ca86aff')}
{'Artist Name': 'Olivia Rodrigo', 'Track Name': 'favorite crime', 'Danceability': 0.369, 'Energy': 0.272, 'Key': 0.147, 'Loudness': 9, 'Acousticness': 0.866, 'Instrumentalness': 0, 'Liveness': 0.147, 'Valence': 0.218, 'Tempo': 172.929, 'Track Reference': 'https://api.spotify.com/v1/tracks/5JCoSi02qi3jJeHdZXMmR8', '_id': ObjectId('60f789a9e867b24c8ca86b00')}
{'Artist Name': 'Bruno Mars', 'Track Name': 'Leave The Door Open', 'Danceability': 0.586, 'Energy': 0.616, 'Key': 0.0927, 'Loudness': 5, 'Acousticness': 0.182, 'Instrumentalness': 0, 'Liveness': 0.0927, 'Valence': 0.719, 'Tempo': 148.088, 'Track Reference': 'https://api.spotif

{'Artist Name': 'Majestic', 'Track Name': 'Rasputin', 'Danceability': 0.758, 'Energy': 0.913, 'Key': 0.193, 'Loudness': 6, 'Acousticness': 0.00364, 'Instrumentalness': 6.92e-05, 'Liveness': 0.193, 'Valence': 0.658, 'Tempo': 128.04, 'Track Reference': 'https://api.spotify.com/v1/tracks/0b18g3G5spr4ZCkz7Y6Q0Q', '_id': ObjectId('60f789ade867b24c8ca86b17')}
{'Artist Name': 'Surf Curse', 'Track Name': 'Freaks', 'Danceability': 0.345, 'Energy': 0.941, 'Key': 0.0511, 'Loudness': 9, 'Acousticness': 3.84e-05, 'Instrumentalness': 0.633, 'Liveness': 0.0511, 'Valence': 0.407, 'Tempo': 179.626, 'Track Reference': 'https://api.spotify.com/v1/tracks/7EkWXAI1wn8Ii883ecd9xr', '_id': ObjectId('60f789ade867b24c8ca86b18')}
{'Artist Name': 'Rauw Alejandro', 'Track Name': '2/Catorce', 'Danceability': 0.768, 'Energy': 0.56, 'Key': 0.115, 'Loudness': 3, 'Acousticness': 0.709, 'Instrumentalness': 0.000791, 'Liveness': 0.115, 'Valence': 0.372, 'Tempo': 112.084, 'Track Reference': 'https://api.spotify.com/v1/tra

{'Artist Name': 'Pop Smoke', 'Track Name': 'Bout A Million (feat. 42 Dugg & 21 Savage)', 'Danceability': 0.716, 'Energy': 0.67, 'Key': 0.13, 'Loudness': 0, 'Acousticness': 0.526, 'Instrumentalness': 0, 'Liveness': 0.13, 'Valence': 0.411, 'Tempo': 145.043, 'Track Reference': 'https://api.spotify.com/v1/tracks/4LaGu95Ui2s4vprSQYWUAZ', '_id': ObjectId('60f789b1e867b24c8ca86b2e')}
{'Artist Name': 'The Kid LAROI', 'Track Name': 'WITHOUT YOU', 'Danceability': 0.662, 'Energy': 0.413, 'Key': 0.134, 'Loudness': 0, 'Acousticness': 0.213, 'Instrumentalness': 0, 'Liveness': 0.134, 'Valence': 0.467, 'Tempo': 93.005, 'Track Reference': 'https://api.spotify.com/v1/tracks/27OeeYzk6klgBh83TSvGMA', '_id': ObjectId('60f789b2e867b24c8ca86b2f')}
{'Artist Name': 'Drake', 'Track Name': 'Wants and Needs (feat. Lil Baby)', 'Danceability': 0.578, 'Energy': 0.449, 'Key': 0.119, 'Loudness': 1, 'Acousticness': 0.0618, 'Instrumentalness': 2.17e-06, 'Liveness': 0.119, 'Valence': 0.1, 'Tempo': 136.006, 'Track Referen

{'Artist Name': 'Internet Money', 'Track Name': 'Lemonade (feat. Gunna, Don Toliver & NAV)', 'Danceability': 0.8, 'Energy': 0.658, 'Key': 0.111, 'Loudness': 1, 'Acousticness': 0.25, 'Instrumentalness': 0, 'Liveness': 0.111, 'Valence': 0.462, 'Tempo': 140.042, 'Track Reference': 'https://api.spotify.com/v1/tracks/7hxHWCCAIIxFLCzvDgnQHX', '_id': ObjectId('60f789b6e867b24c8ca86b45')}
{'Artist Name': 'Bruno Mars', 'Track Name': 'Talking to the Moon', 'Danceability': 0.523, 'Energy': 0.606, 'Key': 0.106, 'Loudness': 1, 'Acousticness': 0.512, 'Instrumentalness': 0, 'Liveness': 0.106, 'Valence': 0.065, 'Tempo': 145.837, 'Track Reference': 'https://api.spotify.com/v1/tracks/161DnLWsx1i3u1JT05lzqU', '_id': ObjectId('60f789b6e867b24c8ca86b46')}
{'Artist Name': 'Shouse', 'Track Name': 'Love Tonight - Edit', 'Danceability': 0.727, 'Energy': 0.681, 'Key': 0.0861, 'Loudness': 5, 'Acousticness': 0.00214, 'Instrumentalness': 0.000307, 'Liveness': 0.0861, 'Valence': 0.448, 'Tempo': 123.028, 'Track Refe

{'Artist Name': 'J Balvin', 'Track Name': 'In Da Getto', 'Danceability': 0.915, 'Energy': 0.72, 'Key': 0.0942, 'Loudness': 8, 'Acousticness': 0.025, 'Instrumentalness': 1.42e-05, 'Liveness': 0.0942, 'Valence': 0.631, 'Tempo': 126.986, 'Track Reference': 'https://api.spotify.com/v1/tracks/63aj87TQG6F3RVO5nbG2VQ', '_id': ObjectId('60f789bae867b24c8ca86b5c')}
{'Artist Name': 'DaBaby', 'Track Name': 'ROCKSTAR (feat. Roddy Ricch)', 'Danceability': 0.746, 'Energy': 0.69, 'Key': 0.101, 'Loudness': 11, 'Acousticness': 0.247, 'Instrumentalness': 0, 'Liveness': 0.101, 'Valence': 0.497, 'Tempo': 89.977, 'Track Reference': 'https://api.spotify.com/v1/tracks/7ytR5pFWmSjzHJIeQkgog4', '_id': ObjectId('60f789bae867b24c8ca86b5d')}
{'Artist Name': 'Normani', 'Track Name': 'Wild Side (feat. Cardi B)', 'Danceability': 0.74, 'Energy': 0.576, 'Key': 0.104, 'Loudness': 6, 'Acousticness': 0.0249, 'Instrumentalness': 0, 'Liveness': 0.104, 'Valence': 0.315, 'Tempo': 110.095, 'Track Reference': 'https://api.spot

{'Artist Name': 'Pop Smoke', 'Track Name': 'What You Know Bout Love', 'Danceability': 0.709, 'Energy': 0.548, 'Key': 0.133, 'Loudness': 10, 'Acousticness': 0.65, 'Instrumentalness': 1.59e-06, 'Liveness': 0.133, 'Valence': 0.543, 'Tempo': 83.995, 'Track Reference': 'https://api.spotify.com/v1/tracks/1tkg4EHVoqnhR6iFEXb60y', '_id': ObjectId('60f789bfe867b24c8ca86b74')}
{'Artist Name': 'Jamie Miller', 'Track Name': "Here's Your Perfect", 'Danceability': 0.558, 'Energy': 0.454, 'Key': 0.146, 'Loudness': 6, 'Acousticness': 0.499, 'Instrumentalness': 0, 'Liveness': 0.146, 'Valence': 0.424, 'Tempo': 113.12, 'Track Reference': 'https://api.spotify.com/v1/tracks/2aTKrdenCq5qBOoJPFdn4P', '_id': ObjectId('60f789bfe867b24c8ca86b75')}
{'Artist Name': 'Ofenbach', 'Track Name': 'Wasted Love (feat. Lagique)', 'Danceability': 0.734, 'Energy': 0.855, 'Key': 0.0901, 'Loudness': 6, 'Acousticness': 0.0821, 'Instrumentalness': 4.73e-06, 'Liveness': 0.0901, 'Valence': 0.54, 'Tempo': 121.955, 'Track Reference

{'Artist Name': 'Powfu', 'Track Name': 'death bed (coffee for your head)', 'Danceability': 0.726, 'Energy': 0.431, 'Key': 0.696, 'Loudness': 8, 'Acousticness': 0.731, 'Instrumentalness': 0, 'Liveness': 0.696, 'Valence': 0.348, 'Tempo': 144.026, 'Track Reference': 'https://api.spotify.com/v1/tracks/7eJMfftS33KTjuF7lTsMCx', '_id': ObjectId('60f789c3e867b24c8ca86b8c')}
{'Artist Name': 'The Killers', 'Track Name': 'Mr. Brightside', 'Danceability': 0.352, 'Energy': 0.911, 'Key': 0.0995, 'Loudness': 1, 'Acousticness': 0.00121, 'Instrumentalness': 0, 'Liveness': 0.0995, 'Valence': 0.236, 'Tempo': 148.033, 'Track Reference': 'https://api.spotify.com/v1/tracks/003vvx7Niy0yvhvHt4a68B', '_id': ObjectId('60f789c4e867b24c8ca86b8d')}
{'Artist Name': 'SZA', 'Track Name': 'Good Days', 'Danceability': 0.436, 'Energy': 0.655, 'Key': 0.688, 'Loudness': 1, 'Acousticness': 0.499, 'Instrumentalness': 8.1e-06, 'Liveness': 0.688, 'Valence': 0.412, 'Tempo': 121.002, 'Track Reference': 'https://api.spotify.com/

{'Artist Name': 'Pop Smoke', 'Track Name': 'Genius (feat. Lil Tjay & Swae Lee)', 'Danceability': 0.723, 'Energy': 0.667, 'Key': 0.261, 'Loudness': 2, 'Acousticness': 0.256, 'Instrumentalness': 0, 'Liveness': 0.261, 'Valence': 0.396, 'Tempo': 143.044, 'Track Reference': 'https://api.spotify.com/v1/tracks/0kNYrbMehFxp1eleTwUjkP', '_id': ObjectId('60f789c8e867b24c8ca86ba4')}
{'Artist Name': 'Ariana Grande', 'Track Name': '7 rings', 'Danceability': 0.778, 'Energy': 0.317, 'Key': 0.0881, 'Loudness': 1, 'Acousticness': 0.592, 'Instrumentalness': 0, 'Liveness': 0.0881, 'Valence': 0.327, 'Tempo': 140.048, 'Track Reference': 'https://api.spotify.com/v1/tracks/6ocbgoVGwYJhOv1GgI9NsF', '_id': ObjectId('60f789c8e867b24c8ca86ba5')}
{'Artist Name': 'Doja Cat', 'Track Name': 'Woman', 'Danceability': 0.824, 'Energy': 0.764, 'Key': 0.117, 'Loudness': 5, 'Acousticness': 0.0888, 'Instrumentalness': 0.00294, 'Liveness': 0.117, 'Valence': 0.881, 'Tempo': 107.998, 'Track Reference': 'https://api.spotify.com